In [1]:
import os
import instructor
from openai import OpenAI
from dotenv import load_dotenv
from services.messages_service import MessagesService

load_dotenv()
client = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

In [2]:
# Initialize vars
messages_service = MessagesService()
openai_model = "gpt-3.5-turbo"
user_interest_enum = None

# Initial system message
system_prompt = """You are an assistant that helps users find sushi restaurants and parking spots in Munich. 
When a user expresses interest in either sushi, parking, or both:
1. Use the set_user_interest function to record their preference
2. Provide relevant parking or restaurant information
3. ALWAYS respond with a natural language message summarizing the information for the user
"""

messages_service.add_system_message(system_prompt)

user_prompt = "I want to park my car"
# user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")
messages_service.add_user_message(user_prompt)

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'I want to park my car'}


In [3]:
from models.intent.user_interest_intent import UserInterest

# Get user interest
intent : UserInterest = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=UserInterest,
)

messages_service.add_assistant_message(intent.assistant_message)
messages_service.add_function_message(UserInterest.__name__, intent.interest)

user_interest_enum = intent.interest

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'I want to park my car'}
{'role': 'assistant', 'content': 'Sure, I will help you find parking spots in Munich.'}
{'role': 'function', 'name': 'UserInterest', 'content': <UserInterestEnum.PARKING: 'PARKING'>}


In [4]:
# user_prompt = "List all parking spots"
user_prompt = "How can i pay for parking?"
messages_service.add_user_message(user_prompt)

### Choose response format based on user intend
Its either Parking or Sushi

In [5]:
from services.user_interest_service import get_user_interest

response_model, intent_service = get_user_interest(user_interest_enum)

intent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=response_model,
)

messages_service.add_assistant_message(intent.assistant_message)
messages_service.print_last_message()
messages_service.add_function_message(intent.__class__.__name__, intent)
messages_service.print_last_message()

result = intent_service.handle_intent(intent)
messages_service.add_function_message(intent_service.handle_intent.__name__, result)
messages_service.print_last_message()

{'role': 'assistant', 'content': 'User is asking about payment methods for parking'}
{'role': 'function', 'name': 'UserParkingIntent', 'content': '{"assistant_message":"User is asking about payment methods for parking","query_types":["payment_info"],"specific_spot_name":null,"duration_hours":null,"height_requirement":null,"location":null,"needs_disabled_access":null,"payment_method_preference":null,"response_detail":"detailed"}'}
{'role': 'function', 'name': 'handle_intent', 'content': '{"data":{"payment_methods":[["CREDIT_CARD_VISA","CASH","CREDIT_CARD_AMEX","DEBIT_MAESTRO","CREDIT_CARD_MASTER"],["CREDIT_CARD_VISA","CASH","CREDIT_CARD_MASTER"],["CASH"]]}}'}


In [6]:
from models.intent.assistant_model import AssistantModel

response : AssistantModel = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=AssistantModel,
)

messages_service.add_assistant_message(response.assistant_message)
messages_service.print_last_message()

{'role': 'assistant', 'content': 'You can pay for parking in Munich using various methods including credit cards (Visa, Amex, Mastercard), debit cards (Maestro), and cash depending on the parking spot. Some spots accept all methods while others may only accept cash. Make sure to have the appropriate payment method ready.'}


### Use it like a chat

In [7]:
from models.intent.user_interest_intent import UserInterest

for i in range(2):
    
    # get user input
    user_prompt = input("User: ")
    messages_service.add_user_message(user_prompt)
    messages_service.print_last_message()


    # Get user interest each time to shrink down the response models
    intent : UserInterest = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=UserInterest,
    )

    messages_service.add_assistant_message(intent.assistant_message)
    messages_service.print_last_message()
    messages_service.add_function_message(UserInterest.__name__, intent.interest)
    messages_service.print_last_message()

    user_interest_enum = intent.interest
    response_model, intent_service = get_user_interest(user_interest_enum)

    # get user intent
    intent = client.chat.completions.create(
        model=openai_model,
        messages=messages_service.get_messages(),
        response_model=response_model,
    )

    messages_service.add_function_message(intent.__class__.__name__, intent)
    messages_service.print_last_message()

    
    # handle intent
    if not isinstance(intent_service, AssistantModel):
        result = intent_service.handle_intent(intent)
        messages_service.add_function_message(intent_service.handle_intent.__name__, result)
        messages_service.print_last_message()

        # Assistant help to summarize the data
        response = client.chat.completions.create(
            model=openai_model,
            messages=messages_service.get_messages(),
            response_model=AssistantModel,
        )

        messages_service.add_assistant_message(response.assistant_message)
        messages_service.print_last_message()


{'role': 'user', 'content': ''}
{'role': 'assistant', 'content': 'User is interested in parking'}
{'role': 'function', 'name': 'UserInterest', 'content': <UserInterestEnum.PARKING: 'PARKING'>}
{'role': 'function', 'name': 'UserParkingIntent', 'content': '{"assistant_message":"User is interested in parking","query_types":["list_all"],"specific_spot_name":null,"duration_hours":null,"height_requirement":null,"location":null,"needs_disabled_access":null,"payment_method_preference":null,"response_detail":"brief"}'}
{'role': 'function', 'name': 'handle_intent', 'content': '{"data":{"all_spots":["Marienplatz","City","Platzl Hotel München"]}}'}
{'role': 'assistant', 'content': 'There are parking spots available in Munich including Marienplatz, City, and Platzl Hotel München.'}
{'role': 'user', 'content': 'how many spots dou you think will still beavailable around 7:00'}
{'role': 'assistant', 'content': 'User is inquiring about the availability of parking spots around 7:00'}
{'role': 'function'

In [8]:
# reset jupiter kernel
# %reset -f